### **Librerias**

In [1]:
import numpy as np
import gzip
import json
import ast
import pandas as pd
import shutil
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\delfi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# ***TRANSFORMACIONES***

# **User reviews**

In [2]:
# Descomprimo el archivo user reviews


# Ruta al archivo comprimido
ruta_comprimida_ur= 'datasets/user_reviews.json.gz'

# Ruta para guardar temporalmente el archivo descomprimido en Colab
ruta_temporal_ur = 'datasets/user_reviews.json'

# Descomprimir el archivo
with gzip.open(ruta_comprimida_ur, 'rb') as f_in:
    with open(ruta_temporal_ur, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
# lista_ur es una lista para almacenar los diccionarios obtenidos del archivo JSON
lista_ur = []

# Abro el archivo con la ruta especificada, utilizando el encoding 'utf-8'
with open(ruta_temporal_ur, encoding='utf-8') as file:
    # Itero sobre cada línea del archivo
    for line in file.readlines():
        # Convierto cada línea (que se asume contiene un diccionario JSON válido) a un diccionario de Python
        # Utilizo ast.literal_eval() para interpretar la línea como una expresión literal de Python
        lista_ur.append(ast.literal_eval(line))

In [4]:
# Guardo el archivo en un DataFrame
df_user_r = pd.DataFrame(lista_ur)
# Expandir los diccionarios en las listas de la columna 'reviews' en nuevas columnas
df_expanded = pd.json_normalize(df_user_r['reviews'].explode())

# Concatenar las columnas expandidas con el DataFrame original y eliminar la columna 'reviews'
df_user_r = pd.concat([df_user_r.drop(columns='reviews'), df_expanded], axis=1)

# ***FEATURE ENGINEERING***

### Limpieza

In [5]:
# convierto la columna review a un string
df_user_r['review'] = df_user_r['review'].astype(str)

In [6]:
# Funcion para eliminar emojis y simbolos
def clean_text(text):
    '''
    Esta función elimina emojis y símbolos no ASCII de un texto dado.

    Argumentos:
    text: El texto del que se eliminarán los emojis y símbolos

    Devuelve:
    cleaned_text: El texto sin emojis ni símbolos no ASCII
    '''
    cleaned_text = re.sub(r'[^\x00-\x7F]+|', '', text)
    return cleaned_text

# for col in columnas:
df_user_r['review'] = df_user_r['review'].apply(clean_text)

In [7]:
# Elimino filas duplicadas basadas en todas las columnas de User Reviews
ur_no_duplicados = df_user_r.drop_duplicates()
df_user_r = ur_no_duplicados

# Elimino valores nulos
df_user_r= df_user_r.dropna()

In [8]:
# Crear instancia del analizador de sentimientos de VADER en NLTK
sia = SentimentIntensityAnalyzer()

# Función para obtener el análisis de sentimiento y asignar valores en la escala dada
def analysis_sentiment_NLTK(text):

    '''
    Realiza el análisis de sentimiento utilizando VADER de NLTK.

    Parámetros:
    text (str): El texto a analizar para determinar su sentimiento.

    Retorna:
    int: El valor de sentimiento asignado según la escala dada:
         - 0 si es 'malo'
         - 1 si es 'neutral' (o si no hay reseña disponible)
         - 2 si es 'positivo'
    '''

    if pd.isnull(text):
        return 1  # Si no hay reseña, asigna valor neutral (1)
    else:
        sentiment = sia.polarity_scores(text)
        # Asigna valores según la escala dada
        if sentiment['compound'] <= -0.05:
            return 0  # Malo
        elif sentiment['compound'] >= 0.05:
            return 2  # Positivo
        else:
            return 1  # Neutral

In [9]:
df_user_r['sentiment_analysis'] = df_user_r['review'].apply(analysis_sentiment_NLTK)

C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\3829969308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_r['sentiment_analysis'] = df_user_r['review'].apply(analysis_sentiment_NLTK)


In [10]:
# Reemplazar todas las ocurrencias de 'Posted' con una cadena vacía en la columna 'posted'
df_user_r['posted'] = df_user_r['posted'].str.replace('Posted ', '').str.replace('Posted', '')

# Reemplazar 'Posted', coma y punto con una cadena vacía en la columna 'posted'
df_user_r['posted'] = df_user_r['posted'].str.replace('Posted ', '').str.replace(',', '').str.replace('.', '')


# Diccionario para mapear los nombres de los meses a números
month_dict = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}

# Función para cambiar los nombres de los meses a números
def convert_month_to_number(date):
    if isinstance(date, str):  # Verificar si es una cadena de texto
        date_parts = date.split()
        for month_name, month_number in month_dict.items():
            if month_name in date_parts:
                date_parts[date_parts.index(month_name)] = month_number
                break
        return ' '.join(date_parts)
    else:
        return date  # Mantener el valor original si no es una cadena de texto}


# Aplicar la función a la columna 'posted'
df_user_r['posted'] = df_user_r['posted'].apply(convert_month_to_number)

# Reemplaza los espacios con /
df_user_r['posted'] = df_user_r['posted'].str.replace(' ', '/')

# Reemplazar los días con un solo dígito por días con dos dígitos
df_user_r['posted'] = df_user_r['posted'].str.replace(r'(\b\d{1}\b)', r'0\1', regex=True)

# permitirá que pandas convierta las fechas válidas y ponga los valores no válidos como NaT (Not a Time)
df_user_r['posted'] = pd.to_datetime(df_user_r['posted'], errors='coerce')

C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\1100473422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_r['posted'] = df_user_r['posted'].str.replace('Posted ', '').str.replace('Posted', '')
C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\1100473422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_r['posted'] = df_user_r['posted'].str.replace('Posted ', '').str.replace(',', '').str.replace('.', '')
C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\1100473422.py:38: SettingWithCopy

C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\1100473422.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_r['posted'] = df_user_r['posted'].str.replace(r'(\b\d{1}\b)', r'0\1', regex=True)
C:\Users\delfi\AppData\Local\Temp\ipykernel_9552\1100473422.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_r['posted'] = pd.to_datetime(df_user_r['posted'], errors='coerce')


In [11]:
df_user_r.head(10)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,evcentric,http://steamcommunity.com/id/evcentric,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
3,doctr,http://steamcommunity.com/id/doctr,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,maplemage,http://steamcommunity.com/id/maplemage,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2
5,Wackky,http://steamcommunity.com/id/Wackky,,2013-11-29,,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,2
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,,NaT,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,,2015-12-04,"Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",2
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,,2014-11-03,,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ...",2
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,,2014-10-15,,263360,No ratings yet,True,"Random drops and random quests, with stat poin...",1


### Guardado

In [12]:
ruta_guardado_csv = 'datasets/user_reviews.csv'

# Guardar el DataFrame en un archivo CSV
df_user_r.to_csv(ruta_guardado_csv, index=False)

# Esto guardará el DataFrame 'df_user_r' en un archivo CSV sin incluir el índice


# ***TRANSFORMACIONES***

# **Steam Games**

In [2]:
# Descomprimo el archivo steam games

# Ruta comprimida
ruta_comprimida_sg = 'datasets/steam_games.json.gz'

# Ruta temporal
ruta_temporal_sg = 'datasets/steam_games.json'

# Descomprimo
with gzip.open(ruta_comprimida_sg, 'rb') as f_in:
    with open(ruta_temporal_sg, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
# Lo guardo en un pd que me permite leerlo
df_steam_g = pd.read_json(ruta_temporal_sg, lines = True)

# ***FEATURE ENGINEERING***

### Limpieza

In [6]:
# Convierto las columnas que son listas a cadenas de texto (str)
df_steam_g['genres'] = df_steam_g['genres'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
df_steam_g['tags'] = df_steam_g['tags'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
df_steam_g['specs'] = df_steam_g['specs'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

# Convierto la columna release_date a formato de fecha
df_steam_g['release_date'] = pd.to_datetime(df_steam_g['release_date'], errors='coerce')


In [7]:
# Elimino las filas donde todos los valores son nulos y reemplazo los valores NaN con None,
#también elimino filas duplicadas

# Eliminar filas donde todos los valores son nulos (None o NaN)
df_steam_g = df_steam_g.dropna(how='all')

# Reemplazar NaN con None
df_steam_g = df_steam_g.where(pd.notna(df_steam_g), None)

# Eliminar filas duplicadas
df_steam_g = df_steam_g.drop_duplicates()

In [8]:
# La columna early_access no estaba devolviendo valores bool como debía, asi que la transformo en uno
df_steam_g['early_access'] = df_steam_g['early_access'].astype(bool)

### Guardado

In [9]:
ruta_guardado_csv = '/datasets/steam_games.csv'

# Guardar el DataFrame en un archivo CSV
df_steam_g.to_csv(ruta_guardado_csv, index=False)

# Esto guardará el DataFrame 'df_steam_games' en un archivo CSV sin incluir el índice

# ***TRANSFORMACIONES***

# **Users Items**

In [4]:
# Descomprimo el archivo user items

# Ruta comprimida
ruta_comprimida_ui = 'datasets/users_items.json.gz'

# Ruta temporal
ruta_temporal_ui = 'datasets/users_items.json'

# Descomprimo
with gzip.open(ruta_comprimida_ui, 'rb') as f_in:
    with open(ruta_temporal_ui, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [6]:
# lista_ui es una lista para almacenar los diccionarios obtenidos del archivo JSON
lista_ui = []

# Abro el archivo con la ruta especificada, utilizando el encoding 'utf-8'
with open(ruta_temporal_ui, encoding='utf-8') as file:
    # Itero sobre cada línea del archivo
    for line in file.readlines():
        # Convierto cada línea (que se asume contiene un diccionario JSON válido) a un diccionario de Python
        # Utilizo ast.literal_eval() para interpretar la línea como una expresión literal de Python
        lista_ui.append(ast.literal_eval(line))

In [ ]:
# Guardo la lista en un DataFrame
df_users_i = pd.DataFrame(lista_ui)

In [ ]:
# Expandir los diccionarios en las listas de la columna 'reviews' en nuevas columnas
df_expand = pd.json_normalize(df_users_i['items'].explode())

# Concatenar las columnas expandidas con el DataFrame original y eliminar la columna 'reviews'
df_users_i = pd.concat([df_users_i.drop(columns='items'), df_expand], axis=1)

# ***FEATURE ENGINEERING***

### Limpieza

In [ ]:
# Eliminar filas duplicadas basadas en todas las columnas del DataFrame
ui_sin_duplicados = df_users_i.drop_duplicates()

# Eliminar nulos
df_users_i = df_users_i.dropna()

In [ ]:
df_users_i.head(10)

### Guardado

In [ ]:
ruta_guardado_csv = 'datasets/users_items.csv'

# Guardar el DataFrame en un archivo CSV
df_users_i.to_csv(ruta_guardado_csv, index=False)

# Esto guardará el DataFrame 'df_users_items' en un archivo CSV sin incluir el índice
